In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import StaleElementReferenceException
import sys

import unittest, time, re,os,threading
from bs4 import BeautifulSoup as bs
from dateutil import parser
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from settings import TWITTER_USERNAME,TWITTER_PASSWORD,GECKO_PATH

In [ ]:
#from https://www.bbc.com/indonesia/trensosial-49880667
hashtagProMahasiswa=["#kpkkuatkorupsiturun", "#KekuasaanDitanganRakyat", "#GejayanMemanggil2", "#diperkosanegara", "#mahasiswabergerak", "#hidupmahasiswa", "#tolakruukuhp", "#tolakrevisiuukpk", "#hiduprakyatindonesia", "#mositidakpercaya", "#reformasidikorupsi", "#stmmelawan", "#stmbergerak", "#gejayanmemanggil", "#indonesiaberduka", "#bebaskananandabadudu", "#BebaskanDandhyLaksono", "#savekpk", "#petisisavekpk","#belajardaripelajar","#rakyatbergerak","#revisiuukpkfornkri", "#dukungrevisiuukpk", "#kpkcengeng", "#tempokacungkpk", "#kpkpatuhaturan", "#kpklebihbaik", "#sayabersamajokowi", "#jokowimendengarrakyat", "#kitadukungjokowi","#percayalangkahjokowi"]
thisHashtag=hashtagProMahasiswa[30]


In [ ]:
wd = Firefox(executable_path=GECKO_PATH)
wd.base_url = "https://twitter.com/login"
wd.maximize_window()
wd.get(wd.base_url)
time.sleep(3)
loginField = wd.find_element_by_class_name("js-username-field")
loginField.clear()
loginField.send_keys(TWITTER_USERNAME)
time.sleep(1)
passwordField = wd.find_element_by_class_name("js-password-field")
passwordField.clear()
passwordField.send_keys(TWITTER_PASSWORD)
time.sleep(1)
passwordField.submit()
time.sleep(10)


In [ ]:
searchField = wd.find_element_by_xpath("//input[@data-testid='SearchBox_Search_Input']")
searchField.clear()
searchField.send_keys(thisHashtag)
time.sleep(1)
searchField.submit()
time.sleep(3)

In [ ]:
loopCounter = 0
lastHeight = wd.execute_script("return document.body.scrollHeight")
arrTweet=[]
while True:
    if loopCounter > 100:
        print(loopCounter)
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    newHeight = wd.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
        print("break di newheight = lastheight")
        break
    lastHeight = newHeight
    loopCounter = loopCounter + 1
    html_source = wd.page_source
    sourcedata = html_source.encode('utf-8')
    soup=bs(sourcedata)
    arr = [x for x in soup.body.findAll('div', attrs={'data-testid':'tweet'})]
    print("temp result : " + str(len(arr)))
    arrTweet.extend(arr)
print('ended at: ' + str(loopCounter))
print("Total : " + str(len(arrTweet)))


In [ ]:
def extract(soup):
    #url,username,time
    getUrlWithStatus = soup.select("a[href*=status]")
    if len(getUrlWithStatus)<=0:
        print("sad")
    else:
        _urlTweet.append(str(getUrlWithStatus[0]["href"]))
        splitStatus = re.split('/',str(getUrlWithStatus[0]["href"]))
        _username.append("@"+splitStatus[1])
        dateTime = getUrlWithStatus[0].find("time")
        if dateTime is not None:
            _time.append(dateTime["datetime"])
        else:
            _time.append('-')

        #type,extrainfo
        if len(soup.find_all(text=re.compile('Replying to'))) == 1:
            _type.append('reply')
            findTarget = soup.find(text=re.compile('Replying to')).parent
            replyingTo = findTarget.find("span")
            if replyingTo is not None:
                _extraInfoTargetUser.append(replyingTo.text)
            else:
                _extraInfoTargetUser.append('-')
            _extraInfoContent.append('-')
        elif len(soup.find_all(text=re.compile('Quote Tweet'))) == 1:
            _type.append('quotereply')
            try:
                _extraInfoContent.append(str(getUrlWithStatus[1]["href"]))
            except IndexError:
                _extraInfoTargetUser.append('-')
                _extraInfoContent.append('-')
            else: 
                _extraInfoContent.append(str(getUrlWithStatus[1]["href"]))
                splitStatus = re.split('/',str(getUrlWithStatus[1]["href"]))
                _extraInfoTargetUser.append('@'+splitStatus[1])
        else:
            _type.append('tweet')
            _extraInfoTargetUser.append('-')
            _extraInfoContent.append('-')

        #reply,retweet,likes
        reply=0
        retweet=0
        likes =0
        try:
            divReply = soup.select("div[data-testid=reply]")[0].text
        except IndexError:
            _reply.append('0')
            reply=0
        else:
            if len(divReply.strip()) <= 0:
                divReply = '0'
                reply=0
            elif 'K' in divReply:
                print("banyak nih reply")
                divReply = divReply.strip('K')
                newReply = float(divReply) * 1000
                divReply = str(newReply)
                divReply = divReply[:-2]
                reply=divReply
            else:
                divReply = divReply.strip()
                reply=divReply
            print("Reply : "+divReply)
            _reply.append(divReply)

        try:
            divRetweet = soup.select("div[data-testid=retweet]")[0].text
        except IndexError:
            retweet = 0
            _retweet.append('0')
        else:
            if len(divRetweet.strip()) <= 0:
                retweet = 0
                divRetweet = '0'
            elif 'K' in divRetweet:
                print("banyak nih retweet")
                divRetweet = divRetweet.strip('K')
                newRetweet = float(divRetweet) * 1000
                divRetweet = str(newRetweet)
                divRetweet = divRetweet[:-2]
                retweet = divRetweet
            else:
                divRetweet = divRetweet.strip()
                retweet = divRetweet
            print("Retweet : "+divRetweet)
            _retweet.append(divRetweet)

        try:
            divLike = soup.select("div[data-testid=like]")[0].text
        except IndexError:
            _like.append('0')
            like=0
        else:
            if len(divLike.strip()) <= 0:
                divLike = '0'
                like=0
            elif 'K' in divLike:
                    print("banyak nih like")
                    divLike = divLike.strip('K')
                    newLike = float(divLike) * 1000
                    divLike = str(newLike)
                    divLike = divLike[:-2]
                    like=divLike
            else:
                divLike = divLike.strip()
                like=divLike
            print("Like : "+divLike)
            _like.append(divLike)
        
        #count max,total,weight
        maxNum = max(float(like),float(retweet),float(reply))
        total = float(like)+float(retweet)+float(reply)
        weight=0
        if maxNum > 0 and total > 0:
            weight=maxNum/total
        _total.append(str(total))
        _max.append(str(maxNum))
        _weight.append(str(weight))
        print("Total : "+str(total))
        print("Max : "+str(maxNum))
        print("Weight : "+str(weight))

        #Content
        hashtag = []
        mention = []
        urls = []
        divContent = soup.select("div.css-901oao.r-hkyrab.r-1qd0xha.r-a023e6.r-16dba41.r-ad9z0x.r-bcqeeo.r-bnwqim.r-qvutc0")
        try:
            textContent = divContent[0].text
        except IndexError:
            _content.append("-")
            _lang.append("-")
            urls.append("-")
            hashtag.append("-")
            mention.append("-")
            _hashtags.append(hashtag)
            _mentions.append(mention)
            _contentUrl.append(urls)
        else:            
            language = divContent[0]["lang"]
            links = divContent[0].select("a")
            for l in links:
                if '#' in l.text:
                    hashtag.append(l.text)
                elif '@' in l.text:
                    mention.append(l.text)
                elif 'photo' not in l["href"]:
                    if not l.find('title'):
                        print("title sad")
                        urls.append("-")
                    else:
                        urls.append(l["title"])

            _content.append(textContent)
            _lang.append(language)
            _hashtags.append(hashtag)
            _mentions.append(mention)
            _contentUrl.append(urls)

In [ ]:
_urlTweet=[]
_username=[]
_time=[]
_type=[]
_reply=[]
_retweet=[]
_like=[]
_extraInfoTargetUser=[]
_extraInfoContent=[]
_content=[]
_lang=[]
_hashtags=[]
_mentions=[]
_contentUrl=[]
_max=[]
_total=[]
_weight=[]
dict = {
    "urlTweet":_urlTweet,
    "username":_username,
    "time":_time,
    "type":_type,
    "reply":_reply,
    "retweet":_retweet,
    "like":_like,
    "extraInfoTargetUser":_extraInfoTargetUser,
    "extraInfoContent":_extraInfoContent,
    "content":_content,
    "lang":_lang,
    "hashtags":_hashtags,
    "mentions":_mentions,
    "contentUrl":_contentUrl,
    "max":_max,
    "total":_total,
    "weight":_weight
} 


In [ ]:
i=0
for x in arrTweet:
    print("Iter : "+str(i))
    sourcedata=x.encode('utf-8')
    soup=bs(sourcedata)
    extract(soup)
    i+=1

In [ ]:
main = pd.DataFrame.from_dict(dict,orient='index').T 
main.to_csv(thisHashtag+'.csv')